<a href="https://colab.research.google.com/github/iskra3138/stable-baselines/blob/main/Transition_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!apt install swig
!pip install stable-baselines[mpi]==2.10.0

CartPoleEnv Class에 step2 추가
- action만을 외부 입력으로 받는 step함수와 달리
- state와 action을 외부 입력으로 받아서 고정된 s_t, a_t에 따라 s_t+1이 어떻게 변화는 지 확인하려 함


In [2]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""

import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np


class CartPoleEnv(gym.Env):
    """
    Description:
        A pole is attached by an un-actuated joint to a cart, which moves along
        a frictionless track. The pendulum starts upright, and the goal is to
        prevent it from falling over by increasing and reducing the cart's
        velocity.

    Source:
        This environment corresponds to the version of the cart-pole problem
        described by Barto, Sutton, and Anderson

    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Cart Position             -4.8                    4.8
        1       Cart Velocity             -Inf                    Inf
        2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        3       Pole Angular Velocity     -Inf                    Inf

    Actions:
        Type: Discrete(2)
        Num   Action
        0     Push cart to the left
        1     Push cart to the right

        Note: The amount the velocity that is reduced or increased is not
        fixed; it depends on the angle the pole is pointing. This is because
        the center of gravity of the pole increases the amount of energy needed
        to move the cart underneath it

    Reward:
        Reward is 1 for every step taken, including the termination step

    Starting State:
        All observations are assigned a uniform random value in [-0.05..0.05]

    Episode Termination:
        Pole Angle is more than 12 degrees.
        Cart Position is more than 2.4 (center of the cart reaches the edge of
        the display).
        Episode length is greater than 200.
        Solved Requirements:
        Considered solved when the average return is greater than or equal to
        195.0 over 100 consecutive trials.
    """

    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates
        self.kinematics_integrator = 'euler'

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds.
        high = np.array([self.x_threshold * 2,
                         np.finfo(np.float32).max,
                         self.theta_threshold_radians * 2,
                         np.finfo(np.float32).max],
                        dtype=np.float32)

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg

        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag if action == 1 else -self.force_mag
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (force + self.polemass_length * theta_dot ** 2 * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0 / 3.0 - self.masspole * costheta ** 2 / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == 'euler':
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}
        
    ##### step2 추가 #######
    def step2(self, state, action):
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action), err_msg

        x, x_dot, theta, theta_dot = state # self.state
        force = self.force_mag if action == 1 else -self.force_mag
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (force + self.polemass_length * theta_dot ** 2 * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0 / 3.0 - self.masspole * costheta ** 2 / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == 'euler':
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            print ('# Pole just fell!')
            print (self.state)
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_done += 1
            reward = 0.0
            print ('# reward = 0')
            print (self.state)

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold * 2
        scale = screen_width/world_width
        carty = 100  # TOP OF CART
        polewidth = 10.0
        polelen = scale * (2 * self.length)
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l, r, t, b = -cartwidth / 2, cartwidth / 2, cartheight / 2, -cartheight / 2
            axleoffset = cartheight / 4.0
            cart = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l, r, t, b = -polewidth / 2, polewidth / 2, polelen - polewidth / 2, -polewidth / 2
            pole = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            pole.set_color(.8, .6, .4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth/2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5, .5, .8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0, carty), (screen_width, carty))
            self.track.set_color(0, 0, 0)
            self.viewer.add_geom(self.track)

            self._pole_geom = pole

        if self.state is None:
            return None

        # Edit the pole polygon vertex
        pole = self._pole_geom
        l, r, t, b = -polewidth / 2, polewidth / 2, polelen - polewidth / 2, -polewidth / 2
        pole.v = [(l, b), (l, t), (r, t), (r, b)]

        x = self.state
        cartx = x[0] * scale + screen_width / 2.0  # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array=mode == 'rgb_array')

    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None

CartPoleEnv_source Class에도 step2 추가
- force_mag = 1000으로 설정

In [8]:
#from gym.envs.classic_control import cartpole

#class CartPoleEnv_source(cartpole.CartPoleEnv):
class CartPoleEnv_source(CartPoleEnv):
    def __init__(self):
        super(CartPoleEnv_source, self).__init__()
        self.max_episode_steps = 500 # v3
        self._elapsed_steps = 0
        # self.gravity = 1.0
        self.force_mag = 1000.0
        # self.polemass_length = self.polemass_length * 0.5

    def step(self, action):
        observation, reward, done, info = super(CartPoleEnv_source, self).step(action)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self.max_episode_steps:
            info['TimeLimit.truncated'] = not done
            done = True
        return observation, reward, done, info
        
    #### step2 추가 #######
    def step2(self, state, action):
        observation, reward, done, info = super(CartPoleEnv_source, self).step2(state, action)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self.max_episode_steps:
            info['TimeLimit.truncated'] = not done
            done = True
        return observation, reward, done, info

    def reset(self):
        self._elapsed_steps = 0
        return super(CartPoleEnv_source, self).reset()

1차 실험 - gym 환경

In [9]:
source = CartPoleEnv_source()
target = CartPoleEnv()

In [13]:
obs = [0.1,0.1,0.1,0.1]
action = 0

In [17]:
for i in range(10) :
  new_obs, reward, done, info = source.step2(obs, action)
  print (new_obs)

[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]
[  0.102      -19.3993984    0.102       29.23232496]


In [18]:
for i in range(10) :
  new_obs, reward, done, info = target.step2(obs, action)
  print (new_obs)

[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]
[ 0.102      -0.09640235  0.102       0.42248276]


2차 실험 - stable-baselines 환경

In [19]:
import gym
import numpy as np

from stable_baselines.common.policies import MlpPolicy
#from stable_baselines.common import make_vec_env
from stable_baselines.common.vec_env import VecNormalize
from stable_baselines import A2C
from stable_baselines import PPO2
from stable_baselines.gail import generate_expert_traj
# from darc import darc_envs

base_vec_env.py에 step2추가

In [20]:
#@title base_vec_env.py
from abc import ABC, abstractmethod
import inspect
import pickle
from typing import Sequence, Optional, List, Union

import cloudpickle
import numpy as np

from stable_baselines import logger
from stable_baselines.common.tile_images import tile_images


class AlreadySteppingError(Exception):
    """
    Raised when an asynchronous step is running while
    step_async() is called again.
    """

    def __init__(self):
        msg = 'already running an async step'
        Exception.__init__(self, msg)


class NotSteppingError(Exception):
    """
    Raised when an asynchronous step is not running but
    step_wait() is called.
    """

    def __init__(self):
        msg = 'not running an async step'
        Exception.__init__(self, msg)


class VecEnv(ABC):
    """
    An abstract asynchronous, vectorized environment.

    :param num_envs: (int) the number of environments
    :param observation_space: (Gym Space) the observation space
    :param action_space: (Gym Space) the action space
    """
    metadata = {
        'render.modes': ['human', 'rgb_array']
    }

    def __init__(self, num_envs, observation_space, action_space):
        self.num_envs = num_envs
        self.observation_space = observation_space
        self.action_space = action_space

    @abstractmethod
    def reset(self):
        """
        Reset all the environments and return an array of
        observations, or a tuple of observation arrays.

        If step_async is still doing work, that work will
        be cancelled and step_wait() should not be called
        until step_async() is invoked again.

        :return: ([int] or [float]) observation
        """
        pass

    @abstractmethod
    def step_async(self, actions):
        """
        Tell all the environments to start taking a step
        with the given actions.
        Call step_wait() to get the results of the step.

        You should not call this if a step_async run is
        already pending.
        """
        pass

    @abstractmethod
    def step2_async(self, state, actions):
        """
        Tell all the environments to start taking a step
        with the given actions.
        Call step_wait() to get the results of the step.

        You should not call this if a step_async run is
        already pending.
        """
        pass

    @abstractmethod
    def step_wait(self):
        """
        Wait for the step taken with step_async().

        :return: ([int] or [float], [float], [bool], dict) observation, reward, done, information
        """
        pass

    @abstractmethod
    def step2_wait(self):
        """
        Wait for the step taken with step_async().

        :return: ([int] or [float], [float], [bool], dict) observation, reward, done, information
        """
        pass

    @abstractmethod
    def close(self):
        """
        Clean up the environment's resources.
        """
        pass

    @abstractmethod
    def get_attr(self, attr_name, indices=None):
        """
        Return attribute from vectorized environment.

        :param attr_name: (str) The name of the attribute whose value to return
        :param indices: (list,int) Indices of envs to get attribute from
        :return: (list) List of values of 'attr_name' in all environments
        """
        pass

    @abstractmethod
    def set_attr(self, attr_name, value, indices=None):
        """
        Set attribute inside vectorized environments.

        :param attr_name: (str) The name of attribute to assign new value
        :param value: (obj) Value to assign to `attr_name`
        :param indices: (list,int) Indices of envs to assign value
        :return: (NoneType)
        """
        pass

    @abstractmethod
    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        """
        Call instance methods of vectorized environments.

        :param method_name: (str) The name of the environment method to invoke.
        :param indices: (list,int) Indices of envs whose method to call
        :param method_args: (tuple) Any positional arguments to provide in the call
        :param method_kwargs: (dict) Any keyword arguments to provide in the call
        :return: (list) List of items returned by the environment's method call
        """
        pass

    @abstractmethod
    def seed(self, seed: Optional[int] = None) -> List[Union[None, int]]:
        """
        Sets the random seeds for all environments, based on a given seed.
        Each individual environment will still get its own seed, by incrementing the given seed.

        :param seed: (Optional[int]) The random seed. May be None for completely random seeding.
        :return: (List[Union[None, int]]) Returns a list containing the seeds for each individual env.
            Note that all list elements may be None, if the env does not return anything when being seeded.
        """
        pass

    def step(self, actions):
        """
        Step the environments with the given action

        :param actions: ([int] or [float]) the action
        :return: ([int] or [float], [float], [bool], dict) observation, reward, done, information
        """
        self.step_async(actions)
        return self.step_wait()

    def step2(self, state, actions):
        """
        Step the environments with the given action

        :param actions: ([int] or [float]) the action
        :return: ([int] or [float], [float], [bool], dict) observation, reward, done, information
        """
        self.step2_async(state, actions)
        return self.step2_wait()

    def get_images(self) -> Sequence[np.ndarray]:
        """
        Return RGB images from each environment
        """
        raise NotImplementedError

    def render(self, mode: str = 'human'):
        """
        Gym environment rendering

        :param mode: the rendering type
        """
        try:
            imgs = self.get_images()
        except NotImplementedError:
            logger.warn('Render not defined for {}'.format(self))
            return

        # Create a big image by tiling images from subprocesses
        bigimg = tile_images(imgs)
        if mode == 'human':
            import cv2  # pytype:disable=import-error
            cv2.imshow('vecenv', bigimg[:, :, ::-1])
            cv2.waitKey(1)
        elif mode == 'rgb_array':
            return bigimg
        else:
            raise NotImplementedError

    @property
    def unwrapped(self):
        if isinstance(self, VecEnvWrapper):
            return self.venv.unwrapped
        else:
            return self

    def getattr_depth_check(self, name, already_found):
        """Check if an attribute reference is being hidden in a recursive call to __getattr__

        :param name: (str) name of attribute to check for
        :param already_found: (bool) whether this attribute has already been found in a wrapper
        :return: (str or None) name of module whose attribute is being shadowed, if any.
        """
        if hasattr(self, name) and already_found:
            return "{0}.{1}".format(type(self).__module__, type(self).__name__)
        else:
            return None

    def _get_indices(self, indices):
        """
        Convert a flexibly-typed reference to environment indices to an implied list of indices.

        :param indices: (None,int,Iterable) refers to indices of envs.
        :return: (list) the implied list of indices.
        """
        if indices is None:
            indices = range(self.num_envs)
        elif isinstance(indices, int):
            indices = [indices]
        return indices


class VecEnvWrapper(VecEnv):
    """
    Vectorized environment base class

    :param venv: (VecEnv) the vectorized environment to wrap
    :param observation_space: (Gym Space) the observation space (can be None to load from venv)
    :param action_space: (Gym Space) the action space (can be None to load from venv)
    """

    def __init__(self, venv, observation_space=None, action_space=None):
        self.venv = venv
        VecEnv.__init__(self, num_envs=venv.num_envs, observation_space=observation_space or venv.observation_space,
                        action_space=action_space or venv.action_space)
        self.class_attributes = dict(inspect.getmembers(self.__class__))

    def step_async(self, actions):
        self.venv.step_async(actions)

    def step2_async(self, state, actions):
        self.venv.step2_async(state, actions)

    @abstractmethod
    def reset(self):
        pass

    @abstractmethod
    def step_wait(self):
        pass

    @abstractmethod
    def step2_wait(self):
        pass

    def seed(self, seed=None):
        return self.venv.seed(seed)

    def close(self):
        return self.venv.close()

    def render(self, mode: str = 'human'):
        return self.venv.render(mode=mode)

    def get_images(self):
        return self.venv.get_images()

    def get_attr(self, attr_name, indices=None):
        return self.venv.get_attr(attr_name, indices)

    def set_attr(self, attr_name, value, indices=None):
        return self.venv.set_attr(attr_name, value, indices)

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        return self.venv.env_method(method_name, *method_args, indices=indices, **method_kwargs)

    def __getattr__(self, name):
        """Find attribute from wrapped venv(s) if this wrapper does not have it.
        Useful for accessing attributes from venvs which are wrapped with multiple wrappers
        which have unique attributes of interest.
        """
        blocked_class = self.getattr_depth_check(name, already_found=False)
        if blocked_class is not None:
            own_class = "{0}.{1}".format(type(self).__module__, type(self).__name__)
            format_str = ("Error: Recursive attribute lookup for {0} from {1} is "
                          "ambiguous and hides attribute from {2}")
            raise AttributeError(format_str.format(name, own_class, blocked_class))

        return self.getattr_recursive(name)

    def _get_all_attributes(self):
        """Get all (inherited) instance and class attributes

        :return: (dict<str, object>) all_attributes
        """
        all_attributes = self.__dict__.copy()
        all_attributes.update(self.class_attributes)
        return all_attributes

    def getattr_recursive(self, name):
        """Recursively check wrappers to find attribute.

        :param name (str) name of attribute to look for
        :return: (object) attribute
        """
        all_attributes = self._get_all_attributes()
        if name in all_attributes:  # attribute is present in this wrapper
            attr = getattr(self, name)
        elif hasattr(self.venv, 'getattr_recursive'):
            # Attribute not present, child is wrapper. Call getattr_recursive rather than getattr
            # to avoid a duplicate call to getattr_depth_check.
            attr = self.venv.getattr_recursive(name)
        else:  # attribute not present, child is an unwrapped VecEnv
            attr = getattr(self.venv, name)

        return attr

    def getattr_depth_check(self, name, already_found):
        """See base class.

        :return: (str or None) name of module whose attribute is being shadowed, if any.
        """
        all_attributes = self._get_all_attributes()
        if name in all_attributes and already_found:
            # this venv's attribute is being hidden because of a higher venv.
            shadowed_wrapper_class = "{0}.{1}".format(type(self).__module__, type(self).__name__)
        elif name in all_attributes and not already_found:
            # we have found the first reference to the attribute. Now check for duplicates.
            shadowed_wrapper_class = self.venv.getattr_depth_check(name, True)
        else:
            # this wrapper does not have the attribute. Keep searching.
            shadowed_wrapper_class = self.venv.getattr_depth_check(name, already_found)

        return shadowed_wrapper_class


class CloudpickleWrapper(object):
    def __init__(self, var):
        """
        Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)

        :param var: (Any) the variable you wish to wrap for pickling with cloudpickle
        """
        self.var = var

    def __getstate__(self):
        return cloudpickle.dumps(self.var)

    def __setstate__(self, obs):
        self.var = cloudpickle.loads(obs)

In [21]:
VecEnv.step2

<function __main__.VecEnv.step2>

dummy_vec_env.py에 step2추가

In [22]:
#@title dummy_vec_env.py
from collections import OrderedDict
from typing import Sequence
from copy import deepcopy

import numpy as np

from stable_baselines.common.vec_env.util import copy_obs_dict, dict_to_obs, obs_space_info


class DummyVecEnv(VecEnv):
    """
    Creates a simple vectorized wrapper for multiple environments, calling each environment in sequence on the current
    Python process. This is useful for computationally simple environment such as ``cartpole-v1``, as the overhead of
    multiprocess or multithread outweighs the environment computation time. This can also be used for RL methods that
    require a vectorized environment, but that you want a single environments to train with.

    :param env_fns: ([callable]) A list of functions that will create the environments
        (each callable returns a `Gym.Env` instance when called).
    """

    def __init__(self, env_fns):
        self.envs = [fn() for fn in env_fns]
        env = self.envs[0]
        VecEnv.__init__(self, len(env_fns), env.observation_space, env.action_space)
        obs_space = env.observation_space
        self.keys, shapes, dtypes = obs_space_info(obs_space)

        self.buf_obs = OrderedDict([
            (k, np.zeros((self.num_envs,) + tuple(shapes[k]), dtype=dtypes[k]))
            for k in self.keys])
        self.buf_dones = np.zeros((self.num_envs,), dtype=np.bool)
        self.buf_rews = np.zeros((self.num_envs,), dtype=np.float32)
        self.buf_infos = [{} for _ in range(self.num_envs)]
        self.actions = None
        self.metadata = env.metadata

    def step_async(self, actions):
        self.actions = actions

    def step2_async(self, state, actions):
        self.actions = actions
        self.state = state

    def step_wait(self):
        for env_idx in range(self.num_envs):
            obs, self.buf_rews[env_idx], self.buf_dones[env_idx], self.buf_infos[env_idx] =\
                self.envs[env_idx].step(self.actions[env_idx])
            if self.buf_dones[env_idx]:
                # save final observation where user can get it, then reset
                self.buf_infos[env_idx]['terminal_observation'] = obs
                obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return (self._obs_from_buf(), np.copy(self.buf_rews), np.copy(self.buf_dones),
                deepcopy(self.buf_infos))
        
    def step2_wait(self):
        for env_idx in range(self.num_envs):
            obs, self.buf_rews[env_idx], self.buf_dones[env_idx], self.buf_infos[env_idx] =\
                self.envs[env_idx].step2(self.state[env_idx], self.actions[env_idx])
            if self.buf_dones[env_idx]:
                # save final observation where user can get it, then reset
                self.buf_infos[env_idx]['terminal_observation'] = obs
                obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return (self._obs_from_buf(), np.copy(self.buf_rews), np.copy(self.buf_dones),
                deepcopy(self.buf_infos))

    def seed(self, seed=None):
        seeds = list()
        for idx, env in enumerate(self.envs):
            seeds.append(env.seed(seed + idx))
        return seeds

    def reset(self):
        for env_idx in range(self.num_envs):
            obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return self._obs_from_buf()

    def close(self):
        for env in self.envs:
            env.close()

    def get_images(self) -> Sequence[np.ndarray]:
        return [env.render(mode='rgb_array') for env in self.envs]

    def render(self, mode: str = 'human'):
        """
        Gym environment rendering. If there are multiple environments then
        they are tiled together in one image via `BaseVecEnv.render()`.
        Otherwise (if `self.num_envs == 1`), we pass the render call directly to the
        underlying environment.

        Therefore, some arguments such as `mode` will have values that are valid
        only when `num_envs == 1`.

        :param mode: The rendering type.
        """
        if self.num_envs == 1:
            return self.envs[0].render(mode=mode)
        else:
            return super().render(mode=mode)

    def _save_obs(self, env_idx, obs):
        for key in self.keys:
            if key is None:
                self.buf_obs[key][env_idx] = obs
            else:
                self.buf_obs[key][env_idx] = obs[key]

    def _obs_from_buf(self):
        return dict_to_obs(self.observation_space, copy_obs_dict(self.buf_obs))

    def get_attr(self, attr_name, indices=None):
        """Return attribute from vectorized environment (see base class)."""
        target_envs = self._get_target_envs(indices)
        return [getattr(env_i, attr_name) for env_i in target_envs]

    def set_attr(self, attr_name, value, indices=None):
        """Set attribute inside vectorized environments (see base class)."""
        target_envs = self._get_target_envs(indices)
        for env_i in target_envs:
            setattr(env_i, attr_name, value)

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        """Call instance methods of vectorized environments."""
        target_envs = self._get_target_envs(indices)
        return [getattr(env_i, method_name)(*method_args, **method_kwargs) for env_i in target_envs]

    def _get_target_envs(self, indices):
        indices = self._get_indices(indices)
        return [self.envs[i] for i in indices]

In [23]:
DummyVecEnv.step2

<function __main__.VecEnv.step2>

In [24]:
#@title make_vec_env
"""
Helpers for scripts like run_atari.py.
"""

import os
import warnings

import gym

from stable_baselines import logger
from stable_baselines.bench import Monitor
from stable_baselines.common.misc_util import set_global_seeds
from stable_baselines.common.atari_wrappers import make_atari, wrap_deepmind
from stable_baselines.common.misc_util import mpi_rank_or_zero
from stable_baselines.common.vec_env import SubprocVecEnv


def make_vec_env(env_id, n_envs=1, seed=None, start_index=0,
                 monitor_dir=None, wrapper_class=None,
                 env_kwargs=None, vec_env_cls=None, vec_env_kwargs=None):
    """
    Create a wrapped, monitored `VecEnv`.
    By default it uses a `DummyVecEnv` which is usually faster
    than a `SubprocVecEnv`.

    :param env_id: (str or Type[gym.Env]) the environment ID or the environment class
    :param n_envs: (int) the number of environments you wish to have in parallel
    :param seed: (int) the initial seed for the random number generator
    :param start_index: (int) start rank index
    :param monitor_dir: (str) Path to a folder where the monitor files will be saved.
        If None, no file will be written, however, the env will still be wrapped
        in a Monitor wrapper to provide additional information about training.
    :param wrapper_class: (gym.Wrapper or callable) Additional wrapper to use on the environment.
        This can also be a function with single argument that wraps the environment in many things.
    :param env_kwargs: (dict) Optional keyword argument to pass to the env constructor
    :param vec_env_cls: (Type[VecEnv]) A custom `VecEnv` class constructor. Default: None.
    :param vec_env_kwargs: (dict) Keyword arguments to pass to the `VecEnv` class constructor.
    :return: (VecEnv) The wrapped environment
    """
    env_kwargs = {} if env_kwargs is None else env_kwargs
    vec_env_kwargs = {} if vec_env_kwargs is None else vec_env_kwargs

    def make_env(rank):
        def _init():
            if isinstance(env_id, str):
                env = gym.make(env_id)
                if len(env_kwargs) > 0:
                    warnings.warn("No environment class was passed (only an env ID) so `env_kwargs` will be ignored")
            else:
                env = env_id(**env_kwargs)
            if seed is not None:
                env.seed(seed + rank)
                env.action_space.seed(seed + rank)
            # Wrap the env in a Monitor wrapper
            # to have additional training information
            monitor_path = os.path.join(monitor_dir, str(rank)) if monitor_dir is not None else None
            # Create the monitor folder if needed
            if monitor_path is not None:
                os.makedirs(monitor_dir, exist_ok=True)
            env = Monitor(env, filename=monitor_path)
            # Optionally, wrap the environment with the provided wrapper
            if wrapper_class is not None:
                env = wrapper_class(env)
            return env
        return _init

    # No custom VecEnv is passed
    if vec_env_cls is None:
        # Default: use a DummyVecEnv
        vec_env_cls = DummyVecEnv

    return vec_env_cls([make_env(i + start_index) for i in range(n_envs)], **vec_env_kwargs)


source/target 환경 정의

In [25]:
source_env = make_vec_env(CartPoleEnv_source, n_envs = 1, seed = 1000)
target_env = make_vec_env(CartPoleEnv, n_envs= 1)

In [26]:
source_env.step2

<bound method VecEnv.step2 of <__main__.DummyVecEnv object at 0x7fc54e34a518>>

Policy_IS
- policy fn.은 deterministic argument로 인해 deterministic fn.으로 할지 probability fn.으로 할지 선택 가능해 보임
- 그런데 왜 그걸 Done값으로 선택하는 지는 잘 모르겠음!!!

In [27]:
import numpy as np

from stable_baselines.common.policies import FeedForwardPolicy

class policy_IS(FeedForwardPolicy):
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **_kwargs):
        super(policy_IS, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse,
                                        feature_extraction="mlp", **_kwargs)

    # q_value가 추가됨
    # state, mask는 RNN 을 위한 것
    def step(self, obs, state=None, mask=None, deterministic=False):
        '''

        :param obs: ([float] or [int]) The current observation of the environment
        :param state: ([float]) The last states (used in recurrent policies)
        :param mask: ([float]) The last masks (used in recurrent policies)
        :param deterministic: (bool) Whether or not to return deterministic actions.
        :return: ([float], [float], [float], [float], [float]) actions, values, states, neglogp, q_value_acted
        '''
        if deterministic:
            action, value, neglogp, q_value = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp, self.q_value],
                                                   {self.obs_ph: obs})
            # action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
            #                                        {self.obs_ph: obs})
        else:
            action, value, neglogp, q_value = self.sess.run([self.action, self.value_flat, self.neglogp, self.q_value],
                                                   {self.obs_ph: obs})
            # action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
            #                                        {self.obs_ph: obs})
        q_value_acted = q_value[np.arange(len(q_value)), action]
        return action, value, self.initial_state, neglogp, q_value_acted
        # return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

In [28]:
#@title weight_calculator.py
import numpy as np
import tensorflow as tf

from stable_baselines.common.input import observation_input
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc


class weight_calculator():
    def __init__(self, sess, obs_space, l_rate, max_grad_norm = 0.5, method = 'Classifier', cnn_kwargs = None):
        self.sess = sess
        self.obs_space = obs_space
        self.max_grad_norm = max_grad_norm
        self.policy_kwargs = {} if cnn_kwargs is None else cnn_kwargs

        # method = 'SR' or 'Classifier'
        if method == 'SR':
            # create tabular
            pass
        elif method == 'Classifier':
            self.setup_classifier(l_rate, self.max_grad_norm, cnn_kwargs)

    def setup_classifier(self, l_rate, max_grad_norm, layers = None, act_fun = tf.tanh, reuse = False, feature_extraction = "mlp", **kwargs):
        with tf.compat.v1.variable_scope("classifier", reuse=reuse):
            # Build observation input with encoding depending on the observation space type
            self.obs_src_ph, processed_obs_src = observation_input(self.obs_space, batch_size=None, name='ob_src_classifier',
                                                              scale=(feature_extraction == "cnn"))
            self.obs_trg_ph, processed_obs_trg = observation_input(self.obs_space, batch_size=None, name='ob_trg_classifier',
                                                              scale=(feature_extraction == "cnn"))

            # # Give noise
            # input_noise = 0.1
            # processed_obs_src = tf.keras.layers.GaussianNoise(input_noise)(processed_obs_src)
            # processed_obs_trg = tf.keras.layers.GaussianNoise(input_noise)(processed_obs_trg)

            classifier_input = tf.concat([processed_obs_src, processed_obs_trg], axis=0)
            src_size = tf.shape(processed_obs_src)[0]
            trg_size = tf.shape(processed_obs_trg)[0]
            y_true = tf.concat([tf.zeros(src_size, dtype=tf.float32), tf.ones(trg_size, dtype=tf.float32)], axis=0) # source == 0, target == 1

            # Build NN
            if layers is None:
                layers = [64, 64]

            if feature_extraction == "cnn":
                classfier_latent = self.nature_cnn(classifier_input, **kwargs)
            else:
                classfier_latent = classifier_input
                for idx, (layer_size) in enumerate(layers):
                    assert isinstance(layer_size, int), "Error: net_arch[-1]['classifier'] must only contain integers."
                    classfier_latent = act_fun(linear(classfier_latent, "classifier{}".format(idx),
                                                      layer_size, init_scale=np.sqrt(2)))

            # # One-dimensional logit
            # logits = tf.squeeze(linear(classfier_latent, 'logit', 1))
            # self.prob = tf.nn.sigmoid(logits) # softmax or sigmoid..?
            #
            # # Define loss function: cross_entropy
            # classifier_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = y_true)
            # classifier_loss = tf.reduce_mean(classifier_loss)
            # # self.generator_acc = tf.reduce_mean(tf.to_float(tf.nn.sigmoid(generator_logits) < 0.5))
            # # self.expert_acc = tf.reduce_mean(tf.to_float(tf.nn.sigmoid(expert_logits) > 0.5))
            # # Build entropy loss
            # entropy = tf.reduce_mean(self.logit_bernoulli_entropy(logits))
            # entropy_loss = -0.001 * entropy
            # self.total_loss = classifier_loss + entropy_loss

            # Two logits
            logits = linear(classfier_latent, 'logit', 2)
            probs = tf.nn.softmax(logits, name="probs")
            # probs = tf.nn.sigmoid(logits, name="probs")
            self.prob = probs[:,1] / probs[:,0]
            self.total_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, probs)

            # Operator for updating classifier
            self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=l_rate)

            self.tvars = tf.trainable_variables("classifier")
            self.gradients = tf.gradients(self.total_loss, self.tvars)
            if max_grad_norm is not None:
                self.gradients, self.norm = tf.clip_by_global_norm(self.gradients, max_grad_norm)
            self.grads = list(zip(self.gradients, self.tvars))
            # update network by getting cumulative gradients
            self.updateModel = self.optimizer.apply_gradients(self.grads)

            # tf.summary.scalar('total_loss_classifier', self.total_loss)
            # tf.summary.scalar('classifier_loss', classifier_loss)

    def update_visitations(self, obs_src, obs_trg):
        feed_dict = {self.obs_src_ph: obs_src, self.obs_trg_ph: obs_trg}
        if self.max_grad_norm is not None:
            _, loss, global_norm = self.sess.run([self.updateModel, self.total_loss, self.norm], feed_dict)
        else:
            _, loss = self.sess.run([self.updateModel, self.total_loss], feed_dict)
            global_norm = 0
        return loss, global_norm

    def get_IS(self, obs_src):
        feed_dict = {self.obs_src_ph: obs_src, self.obs_trg_ph: obs_src}
        return self.sess.run(self.prob, feed_dict)

    def nature_cnn(self, scaled_images, **kwargs):
        """
        CNN from Nature paper.

        :param scaled_images: (TensorFlow Tensor) Image input placeholder
        :param kwargs: (dict) Extra keywords parameters for the convolutional layers of the CNN
        :return: (TensorFlow Tensor) The CNN output layer
        """
        activ = tf.nn.relu
        layer_1 = activ(conv(scaled_images, 'c1', n_filters=32, filter_size=8, stride=4, init_scale=np.sqrt(2), **kwargs))
        layer_2 = activ(conv(layer_1, 'c2', n_filters=64, filter_size=4, stride=2, init_scale=np.sqrt(2), **kwargs))
        layer_3 = activ(conv(layer_2, 'c3', n_filters=64, filter_size=3, stride=1, init_scale=np.sqrt(2), **kwargs))
        layer_3 = conv_to_fc(layer_3)
        return activ(linear(layer_3, 'fc1', n_hidden=512, init_scale=np.sqrt(2)))

    def logsigmoid(self, a):
        '''Equivalent to tf.log(tf.sigmoid(a))'''
        return -tf.nn.softplus(-a)

    """ Reference: https://github.com/openai/imitation/blob/99fbccf3e060b6e6c739bdf209758620fcdefd3c/policyopt/thutil.py#L48-L51"""

    def logit_bernoulli_entropy(self, logits):
        ent = (1. - tf.nn.sigmoid(logits)) * logits - self.logsigmoid(logits)
        return ent

A2C_IS
- colab에 설치된 stable-baseline에서는 argument에 momentum이 있으면 에러가 나서 제거함

In [29]:
#@title A2C_IS.py
import time
import tensorflow as tf
import numpy as np
import gym
from gym.spaces.space import Space
from gym.spaces.box import Box
from gym.spaces.discrete import Discrete

from stable_baselines import A2C
from stable_baselines import logger
from stable_baselines.common import explained_variance, tf_util, ActorCriticRLModel, SetVerbosity, TensorboardWriter
from stable_baselines.common.runners import AbstractEnvRunner
from stable_baselines.common.schedules import Scheduler
from stable_baselines.common.tf_util import mse, total_episode_reward_logger
from stable_baselines.common.math_util import safe_mean
from stable_baselines.common.vec_env import VecNormalize, unwrap_vec_normalize
from stable_baselines.common.base_class import _UnvecWrapper
from stable_baselines.common.policies import ActorCriticPolicy, RecurrentActorCriticPolicy
# from stable_baselines.common.buffers import ReplayBuffer

#from A2C_IS_Runner import A2C_IS_Runner
#from weight_calculator import weight_calculator


class A2C_IS(A2C):

    def __init__(self, policy, env, source_env = None,source_demo = None, gamma=0.99, n_steps=5, vf_coef=0.25, ent_coef=0.01, max_grad_norm=0.5,
                 learning_rate=7e-4, alpha=0.99, epsilon=1e-5, lr_schedule='constant',
                 verbose=0, tensorboard_log=None, _init_setup_model=True, policy_kwargs=None,
                 full_tensorboard_log=False, seed=None, n_cpu_tf_sess=None):

        # # Initialize evaluation environment
        # self.eval_env = eval_env

        # Initialize source_env
        self.source_env = source_env
        self.source_demo = source_demo
        # source optimal policy를 활용할 때 사용하는 if문
        if self.source_demo is not None:
            self.threshold = 0.5 # dynamics가 어느 정도 다를 때 안쓸 것인가? 쓸 것인가?
            # demonstration 저장된 거 불러와서 읽어들이는 부분
            self.actions_demo = source_demo['actions']
            self.starts_demo = source_demo['episode_starts']
            self.obs_demo = source_demo['obs']
            self.rewards_demo = source_demo['rewards']
            # temp2 = source_demo['episode_returns']
            self.n_demo_samples = np.shape(self.obs_demo)[0]
            # self.src_buffer = ReplayBuffer(np.shape(self.obs_demo)[0])
        self._source_runner = None

        self.max_grad_norm = max_grad_norm
        self.learning_rate_c = learning_rate
        # self.learning_rate_c = 0.001
        np.random.seed(seed)

        super(A2C_IS, self).__init__(policy=policy, env=env, gamma=gamma, n_steps=n_steps, vf_coef=vf_coef, ent_coef=ent_coef, max_grad_norm=max_grad_norm,
                     learning_rate=learning_rate, alpha=alpha, epsilon=epsilon, lr_schedule=lr_schedule,
                     verbose=verbose, tensorboard_log=tensorboard_log, _init_setup_model=_init_setup_model, policy_kwargs=policy_kwargs,
                     full_tensorboard_log=full_tensorboard_log, seed=seed, n_cpu_tf_sess=n_cpu_tf_sess)
    # Target sample 만드는 부분
    def _make_runner(self) -> AbstractEnvRunner:
        return A2C_IS_Runner(self.env, self, run_target=True, n_steps=self.n_steps, gamma=self.gamma)

    @property
    def source_runner(self) -> AbstractEnvRunner:
        if self._source_runner is None:
            self._source_runner = self._make_source_runner()
        return self._source_runner
    # Source sample 만드는 부분
    def _make_source_runner(self) -> AbstractEnvRunner:
        return A2C_IS_Runner(self.source_env, self, run_target=False, n_steps=self.n_steps, gamma=self.gamma)

    def learn(self, total_timesteps, callback=None, log_interval=100, tb_log_name="A2C",
              reset_num_timesteps=True):

        new_tb_log = self._init_num_timesteps(reset_num_timesteps)
        callback = self._init_callback(callback)

        with SetVerbosity(self.verbose), TensorboardWriter(self.graph, self.tensorboard_log, tb_log_name, new_tb_log) \
                as writer:
            self._setup_learn()
            self.learning_rate_schedule = Scheduler(initial_value=self.learning_rate, n_values=total_timesteps,
                                                    schedule=self.lr_schedule)

            t_start = time.time()
            callback.on_training_start(locals(), globals())

            # num_target_samples = 0

            for update in range(1, total_timesteps // self.n_batch + 1):

                callback.on_rollout_start()
                # true_reward is the reward without discount
                rollout = self.runner.run(callback)
                # unpack
                obs_t, states_t, rewards_t, masks_t, actions_t, values_t, q_values_t, ep_infos, true_reward_t, steps = rollout

                # num_target_samples += np.size(rewards_t)

                if self.source_env is not None: # Integrate source samples
                    rollout_source = self.source_runner.run(callback)
                    obs_s, states_s, rewards_s, masks_s, actions_s, values_s, q_values_s, ep_infos_s, true_reward_s, steps = rollout_source
                    n_source_samples = np.shape(obs_s)[0]
                    n_opt_samples = n_source_samples

                    if self.source_demo is not None:
                        obs_s_opt, n_obs_s_opt, rewards_s_opt, dones_opt, actions_s_opt, values_s_opt = [], [], [], [], [], []
                        # idx = 0
                        n_opt_samples = int(n_source_samples * np.power(0.999, update))
                        # print(n_opt_samples)
                        for n in range(n_opt_samples):
                            idx = np.random.randint(self.n_demo_samples)
                            obs_s_opt.append(self.obs_demo[idx])
                            rewards_s_opt.append(self.rewards_demo[idx])
                            actions_s_opt.append(self.actions_demo[idx])
                            if idx == self.n_demo_samples -1:
                                dones_opt.append(True)
                                n_obs_s_opt.append(self.obs_demo[idx])
                            else:
                                dones_opt.append(self.starts_demo[idx + 1])
                                n_obs_s_opt.append(self.obs_demo[idx + 1])
                            # idx += 1
                        if n_opt_samples is not 0:
                            obs_s_opt = np.asarray(obs_s_opt, dtype=self.observation_space.dtype).swapaxes(1, 0).reshape((n_opt_samples,) + self.observation_space.shape)
                            actions_s_opt = np.asarray(actions_s_opt, dtype=self.action_space.dtype)
                            rewards_s_opt = np.asarray(rewards_s_opt, dtype=np.float32)
                            dones_opt = np.asarray(dones_opt, dtype=np.bool)

                            # Rewards, Value computation
                            _, value, _, _ = self.step(obs = np.append(obs_s_opt, n_obs_s_opt, axis = 0))
                            values_s_opt = value[:n_opt_samples]
                            for i in range(len(rewards_s_opt)):
                                rewards_s_opt[i] = rewards_s_opt[i] + self.gamma * value[n_opt_samples + i] * (1. - dones_opt[i])
                            # Integrate inputs
                            obs_classifier = np.append(obs_s, obs_s_opt, axis = 0)
                        else:
                            obs_classifier = obs_s
                    else:
                        obs_classifier = obs_s

                    # Compute importance weight
                    # input: obs, obs_s, output: weight
                    # weights = np.nan_to_num(probs_to_be_trg / (1-probs_to_be_trg), nan = 1.7976931348623157e+308)
                    # probs_to_be_trg = np.float64(probs_to_be_trg)
                    probs_to_be_trg = self.classifier.get_IS(obs_src=obs_classifier)[0:np.shape(obs_classifier)[0]]

                    # With 1-D logit classifier
                    # with np.errstate(divide='raise'):
                    #     try:
                    #         weights = probs_to_be_trg / (1-probs_to_be_trg)
                    #     except:
                    #         probs_to_be_trg[probs_to_be_trg == 1.0] = 1.0 - 1.0e-10

                    # Without IS
                    # weights = np.ones_like(probs_to_be_trg)

                    # With 2-D logit classifier
                    weights = probs_to_be_trg # two-dimensional logit

                    # Weight clipped by 1
                    # weights = np.minimum(weights, np.ones_like(weights))

                    # # Case when doesn't use estimated ADV
                    # weighted_rewards = rewards_s * weights[0:n_source_samples]
                    # weighted_values = values_s * weights[0:n_source_samples]

                    # Case when use estimated ADV
                    # weighted_rewards = q_values_s * weights[0:n_source_samples]
                    # weighted_values = np.zeros_like(weighted_rewards)
                    weighted_rewards = rewards_s
                    weighted_values = values_s
                    q_values_s = q_values_s * weights[0:n_source_samples]

                    # Threshold..
                    if self.source_demo is not None and n_opt_samples is not 0:
                        delete_rows = []
                        for i in range(n_opt_samples):
                            if np.abs(weights[n_source_samples + i]-1) > self.threshold:
                                delete_rows.append(i)
                        obs_s_opt = np.delete(obs_s_opt,delete_rows,axis=0)
                        actions_s_opt = np.delete(actions_s_opt, delete_rows, axis=0)
                        rewards_s_opt = np.delete(rewards_s_opt[:,0] * weights[n_source_samples:], delete_rows, axis=0)
                        values_s_opt = np.delete(values_s_opt * weights[n_source_samples:], delete_rows, axis=0)

                        obs_s = np.append(obs_s, obs_s_opt, axis = 0)
                        actions_s = np.append(actions_s, actions_s_opt)
                        weighted_rewards = np.append(weighted_rewards, rewards_s_opt)
                        weighted_values = np.append(weighted_values, values_s_opt)

                        # Test case: use only opt src
                        # obs_s = obs_s_opt
                        # actions_s = actions_s_opt
                        # weighted_rewards = rewards_s_opt
                        # weighted_values = values_s_opt

                    # Integrate inputs (target + source)
                    obs = np.append(obs_t, obs_s, axis = 0)
                    masks = np.append(masks_t, masks_s)
                    actions = np.append(actions_t, actions_s)
                    if states_t is not None:
                        print("RNN policy is not applicable")
                    else:
                        states = None
                    rewards = np.append(rewards_t, weighted_rewards)
                    values = np.append(values_t, weighted_values)
                    q_values = np.append(q_values_t, q_values_s)
                    # ep_infos = np.append(ep_infos_t, ep_infos_s)
                    # true_reward = np.append(true_reward_t, true_reward_s)

                else:
                    obs, states, rewards, masks, actions, values, q_values, ep_infos, true_reward_t, steps = rollout

                callback.update_locals(locals())
                callback.on_rollout_end()

                # Early stopping due to the callback
                if not self.runner.continue_training:
                    break

                self.ep_info_buf.extend(ep_infos)

                # Update classifer, actor, critic
                if self.source_env is not None:
                    classifier_loss, _ = self.classifier.update_visitations(obs_s[0:n_source_samples], obs_t)
                else:
                    classifier_loss = 0
                _, value_loss, policy_entropy = self._train_step(obs, states, rewards, masks, actions, values, q_values, np.shape(rewards_t)[0],
                                                                 self.num_timesteps // self.n_batch, writer)
                # # Update value also with source samples
                # _, value_loss, policy_entropy = self._train_step(obs, states, rewards, masks, actions, values, q_values, np.shape(obs)[0],
                #                                                  self.num_timesteps // self.n_batch, writer)

                # Save logs
                n_seconds = time.time() - t_start
                fps = int((update * self.n_batch) / n_seconds)

                if writer is not None:
                    # Evaluation per update
                    # if num_target_samples%(self.n_batch*self.n_envs) == 0:
                    # if num_target_samples % 1280 == 0:
                    #     episodes_to_eval_per_update = 10
                    #     returns = 0
                    #     for i in range(episodes_to_eval_per_update):
                    #         obs_eval = self.eval_env.reset()
                    #         dones = False
                    #         cum_rewards = 0
                    #         while not dones:
                    #             action, _states = self.predict_customize(obs_eval)
                    #             obs_eval, rewards_eval, dones, _ = self.eval_env.step(action)
                    #             cum_rewards += rewards_eval
                    #         returns += cum_rewards
                    #     returns = returns / episodes_to_eval_per_update
                    #
                    #     with tf.variable_scope("evaluation", reuse=True):
                    #         summary = tf.Summary(
                    #             value=[tf.compat.v1.Summary.Value(tag="episode reward (evaluation)", simple_value=returns[0])])
                    #         writer.add_summary(summary, self.num_timesteps)

                    # Original method of obtaining episode reward: Record whenever episode ended
                    total_episode_reward_logger(self.episode_reward,
                                                true_reward_t.reshape((self.n_envs, self.n_steps)),
                                                masks_t.reshape((self.n_envs, self.n_steps)),
                                                writer, self.num_timesteps)
                    if self.source_env is not None:
                        with tf.variable_scope("classifier_info", reuse=True):
                            summary = tf.Summary(value=[tf.compat.v1.Summary.Value(tag="weights", simple_value=np.mean(weights))])
                            writer.add_summary(summary, self.num_timesteps)
                            summary2 = tf.Summary(value=[tf.compat.v1.Summary.Value(tag="classifier_loss", simple_value=np.mean(classifier_loss))])
                            writer.add_summary(summary2, self.num_timesteps)

                if self.verbose >= 1 and (update % log_interval == 0 or update == 1):
                    explained_var = explained_variance(values, rewards)
                    logger.record_tabular("nupdates", update)
                    logger.record_tabular("total_timesteps", self.num_timesteps)
                    logger.record_tabular("fps", fps)
                    logger.record_tabular("policy_entropy", float(policy_entropy))
                    logger.record_tabular("value_loss", float(value_loss))
                    if self.source_env is not None:
                        logger.record_tabular("classifier_loss", float(np.mean(classifier_loss)))
                        logger.record_tabular("mean_weight", float(np.mean(weights)))
                    logger.record_tabular("explained_variance", float(explained_var))
                    if len(self.ep_info_buf) > 0 and len(self.ep_info_buf[0]) > 0:
                        logger.logkv('ep_reward_mean', safe_mean([ep_info['r'] for ep_info in self.ep_info_buf]))
                        logger.logkv('ep_len_mean', safe_mean([ep_info['l'] for ep_info in self.ep_info_buf]))
                    logger.dump_tabular()

            # print("Total target samples: ", num_target_samples)

        callback.on_training_end()
        return self

    def setup_model(self):
        with SetVerbosity(self.verbose):

            assert issubclass(self.policy, ActorCriticPolicy), "Error: the input policy for the A2C model must be an " \
                                                                "instance of common.policies.ActorCriticPolicy."

            self.graph = tf.Graph()
            with self.graph.as_default():
                self.set_random_seed(self.seed)
                self.sess = tf_util.make_session(num_cpu=self.n_cpu_tf_sess, graph=self.graph)

                self.n_batch = self.n_envs * self.n_steps

                n_batch_step = None
                n_batch_train = None
                if issubclass(self.policy, RecurrentActorCriticPolicy):
                    n_batch_step = self.n_envs
                    n_batch_train = self.n_envs * self.n_steps

                # Build classifier
                if self.source_env is not None:
                    classifier = weight_calculator(sess = self.sess, obs_space = self.observation_space, l_rate=self.learning_rate_c,
                                                   max_grad_norm=self.max_grad_norm, **self.policy_kwargs)

                step_model = self.policy(self.sess, self.observation_space, self.action_space, self.n_envs, 1,
                                         n_batch_step, reuse=False, **self.policy_kwargs)

                with tf.variable_scope("train_model", reuse=True,
                                       custom_getter=tf_util.outer_scope_getter("train_model")):
                    train_model = self.policy(self.sess, self.observation_space, self.action_space, self.n_envs,
                                              self.n_steps, n_batch_train, reuse=True, **self.policy_kwargs)

                with tf.variable_scope("loss", reuse=False):
                    self.actions_ph = train_model.pdtype.sample_placeholder([None], name="action_ph")
                    self.advs_ph = tf.placeholder(tf.float32, [None], name="advs_ph")
                    self.rewards_ph = tf.placeholder(tf.float32, [None], name="rewards_ph")
                    self.learning_rate_ph = tf.placeholder(tf.float32, [], name="learning_rate_ph")

                    neglogpac = train_model.proba_distribution.neglogp(self.actions_ph)
                    self.entropy = tf.reduce_mean(train_model.proba_distribution.entropy())
                    self.pg_loss = tf.reduce_mean(self.advs_ph * neglogpac)

                    # self.obs_value_in = tf.slice(tf.squeeze(train_model.value_flat), [0], [tf.shape(self.rewards_ph)[0]])
                    # self.vf_loss = mse(tf.squeeze(train_model.value_flat), self.rewards_ph) # Original, Estimate state-value

                    # # Original
                    # self.vf_loss = mse(tf.slice(tf.squeeze(train_model.value_flat), [0], [tf.shape(self.rewards_ph)[0]]), self.rewards_ph) # Cut-off source samples
                    # loss = self.pg_loss - self.entropy * self.ent_coef + self.vf_loss * self.vf_coef

                    # Dueling network
                    if isinstance(self.action_space, Discrete): # discrete action space
                        q_estimate = train_model.value_flat + tf.reduce_sum(train_model.q_value * tf.one_hot(self.actions_ph, self.action_space.n), axis = 1) \
                                     - tf.reduce_mean(train_model.q_value, axis=-1)
                    else: # continuous
                        pass
                    self.vf_loss = mse(tf.slice(tf.squeeze(q_estimate), [0], [tf.shape(self.rewards_ph)[0]]), self.rewards_ph) # Estimate (state,action)-value
                    # https://arxiv.org/pdf/1708.04782.pdf#page=9, https://arxiv.org/pdf/1602.01783.pdf#page=4
                    # and https://github.com/dennybritz/reinforcement-learning/issues/34
                    # suggest to add an entropy component in order to improve exploration.
                    loss = self.pg_loss - self.entropy * self.ent_coef + self.vf_loss * self.vf_coef # Original

                    # To check unbiased PG
                    params_test = tf_util.get_trainable_vars("model")
                    pg_loss_trg = tf.reduce_mean(tf.slice(self.advs_ph * neglogpac, [0], [tf.shape(self.rewards_ph)[0]]))
                    pg_loss_src = tf.reduce_mean(tf.slice(self.advs_ph * neglogpac, [tf.shape(self.rewards_ph)[0]], [tf.shape(self.advs_ph)[0] - tf.shape(self.rewards_ph)[0]]))

                    # pg_loss_trg = tf.reduce_mean(tf.slice(self.advs_ph * neglogpac, [0], [tf.shape(self.advs_ph)[0]/2]))
                    # pg_loss_src = tf.reduce_mean(tf.slice(self.advs_ph * neglogpac, [10], [tf.shape(self.advs_ph)[0]/2]))

                    pg_grads_trg = tf.gradients(pg_loss_trg, params_test)
                    pg_grads_src = tf.gradients(pg_loss_src, params_test)
                    vf_grads = tf.gradients(self.vf_loss, params_test)

                    _, pg_grads_trg = tf.clip_by_global_norm(pg_grads_trg, 0.5)
                    _, pg_grads_src = tf.clip_by_global_norm(pg_grads_src, 0.5)
                    _, vf_grads = tf.clip_by_global_norm(vf_grads, 0.5)

                    with tf.variable_scope("grad_info", reuse=False):
                        tf.summary.scalar('pg_loss_trg', tf.reduce_mean(pg_loss_trg))
                        tf.summary.scalar('pg_loss_src', tf.reduce_mean(pg_loss_src))
                        tf.summary.scalar('pg_grads_trg', tf.reduce_mean(pg_grads_trg))
                        tf.summary.scalar('pg_grads_src', tf.reduce_mean(pg_grads_src))
                        tf.summary.scalar('vf_grads', tf.reduce_mean(vf_grads))

                    tf.summary.scalar('entropy_loss', self.entropy)
                    tf.summary.scalar('policy_gradient_loss', self.pg_loss)
                    tf.summary.scalar('value_function_loss', self.vf_loss)

                    # Original
                    tf.summary.scalar('loss', loss)
                    self.params = tf_util.get_trainable_vars("model")
                    grads = tf.gradients(loss, self.params)
                    if self.max_grad_norm is not None:
                        grads, _ = tf.clip_by_global_norm(grads, self.max_grad_norm)
                    grads = list(zip(grads, self.params))

                    # # Update policy
                    # tf.summary.scalar('loss_pg', loss_p)
                    # self.params = tf_util.get_trainable_vars("model")
                    # grads_p = tf.gradients(loss_p, self.params)
                    # if self.max_grad_norm is not None:
                    #     grads_p, _ = tf.clip_by_global_norm(grads_p, self.max_grad_norm)
                    # grads_p = list(zip(grads_p, self.params))
                    #
                    # # Update value
                    # tf.summary.scalar('loss_vf', loss_v)
                    # grads_v = tf.gradients(loss_v, self.params)
                    # if self.max_grad_norm is not None:
                    #     grads_v, _ = tf.clip_by_global_norm(grads_v, self.max_grad_norm)
                    # grads_v = list(zip(grads_v, self.params))

                with tf.variable_scope("input_info", reuse=False):
                    tf.summary.scalar('discounted_rewards', tf.reduce_mean(self.rewards_ph))
                    tf.summary.scalar('learning_rate', tf.reduce_mean(self.learning_rate_ph))
                    tf.summary.scalar('advantage', tf.reduce_mean(self.advs_ph))
                    if self.full_tensorboard_log:
                        tf.summary.histogram('discounted_rewards', self.rewards_ph)
                        tf.summary.histogram('learning_rate', self.learning_rate_ph)
                        tf.summary.histogram('advantage', self.advs_ph)
                        if tf_util.is_image(self.observation_space):
                            tf.summary.image('observation', train_model.obs_ph)
                        else:
                            tf.summary.histogram('observation', train_model.obs_ph)

                trainer = tf.compat.v1.train.RMSPropOptimizer(learning_rate=self.learning_rate_ph, decay=self.alpha,
                                                    epsilon=self.epsilon)
                self.apply_backprop = trainer.apply_gradients(grads) # Original
                # self.apply_backprop_p = trainer.apply_gradients(grads_p) # policy update
                # self.apply_backprop_v = trainer.apply_gradients(grads_v) # value update

                self.train_model = train_model
                self.step_model = step_model
                if self.source_env is not None:
                    self.classifier = classifier
                self.step = step_model.step
                self.proba_step = step_model.proba_step
                self.value = step_model.value
                self.initial_state = step_model.initial_state
                tf.global_variables_initializer().run(session=self.sess)

                self.summary = tf.summary.merge_all()

    def _train_step(self, obs, states, rewards, masks, actions, values, q_values, n_sources, update, writer=None):
        """
        applies a training step to the model

        :param obs: ([float]) The input observations
        :param states: ([float]) The states (used for recurrent policies)
        :param rewards: ([float]) The rewards from the environment
        :param masks: ([bool]) Whether or not the episode is over (used for recurrent policies)
        :param actions: ([float]) The actions taken
        :param values: ([float]) The logits values
        :param update: (int) the current step iteration
        :param writer: (TensorFlow Summary.writer) the writer for tensorboard
        :return: (float, float, float) policy loss, value loss, policy entropy
        """
        # State-value based
        # advs = rewards - values # with baseline
        # advs = rewards # without baseline
        advs = q_values

        # (state,action)-value based
        # advs = q_values - values
        # advs = q_values

        cur_lr = None
        for _ in range(n_sources):
            cur_lr = self.learning_rate_schedule.value()
        assert cur_lr is not None, "Error: the observation input array cannon be empty"

        td_map = {self.train_model.obs_ph: obs, self.actions_ph: actions, self.advs_ph: advs,
                  self.rewards_ph: rewards[0:n_sources], self.learning_rate_ph: cur_lr}
        if states is not None:
            td_map[self.train_model.states_ph] = states
            td_map[self.train_model.dones_ph] = masks

        if writer is not None:
            # run loss backprop with summary, but once every 10 runs save the metadata (memory, compute time, ...)
            if self.full_tensorboard_log and (1 + update) % 10 == 0:
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, policy_loss, value_loss, policy_entropy, _ = self.sess.run(
                    [self.summary, self.pg_loss, self.vf_loss, self.entropy, self.apply_backprop],
                    td_map, options=run_options, run_metadata=run_metadata) # test_purpose to track gradient values
                writer.add_run_metadata(run_metadata, 'step%d' % (update * self.n_batch))
            else:
                summary, policy_loss, value_loss, policy_entropy, _ = self.sess.run(
                    [self.summary, self.pg_loss, self.vf_loss, self.entropy, self.apply_backprop], td_map)  # test_purpose to track gradient values
            writer.add_summary(summary, update * self.n_batch)

        else:
            policy_loss, value_loss, policy_entropy, _ = self.sess.run(
                [self.pg_loss, self.vf_loss, self.entropy, self.apply_backprop], td_map)

        return policy_loss, value_loss, policy_entropy

    def predict_customize(self, observation, state=None, mask=None, deterministic=False):
        if state is None:
            state = self.initial_state
        if mask is None:
            mask = [False for _ in range(self.n_envs)]
        observation = np.array(observation)
        vectorized_env = self._is_vectorized_observation(observation, self.observation_space)

        observation = observation.reshape((-1,) + self.observation_space.shape)
        actions, _, states, _, _ = self.step(observation, state, mask, deterministic=deterministic)

        clipped_actions = actions
        # Clip the actions to avoid out of bound error
        if isinstance(self.action_space, gym.spaces.Box):
            clipped_actions = np.clip(actions, self.action_space.low, self.action_space.high)

        if not vectorized_env:
            if state is not None:
                raise ValueError("Error: The environment must be vectorized when using recurrent policies.")
            clipped_actions = clipped_actions[0]

        return clipped_actions, states

In [30]:
model = A2C_IS(policy_IS, target_env, source_env, source_demo=None, ent_coef=0.0, verbose=1, tensorboard_log="./test_for_metric/", seed=0, n_cpu_tf_sess=1)

Wrapping the env in a DummyVecEnv.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


A2C_Runner 코드 일부로 샘플링 테스트

In [32]:
#obs = source_env.reset()
obs = np.array([[0.1,0.1,0.1,0.1]])
#actions, values, states, _, q_values = self.model.step(self.obs, self.states, self.dones) # Get action, values from policy model
actions, values, states, _, q_values = model.step(obs, True) # Get action, values from policy model
clipped_actions = actions
print (obs, clipped_actions)

[[0.1 0.1 0.1 0.1]] [0]


In [33]:
# Source Transition
for i in range(10) :
  #obs, rewards, dones, infos = self.env.step(clipped_actions)
  new_obs, rewards, dones, infos = source_env.step2(obs, clipped_actions)
  print (i, new_obs)

0 [[  0.102    -19.399399   0.102     29.232325]]
1 [[  0.102    -19.399399   0.102     29.232325]]
2 [[  0.102    -19.399399   0.102     29.232325]]
3 [[  0.102    -19.399399   0.102     29.232325]]
4 [[  0.102    -19.399399   0.102     29.232325]]
5 [[  0.102    -19.399399   0.102     29.232325]]
6 [[  0.102    -19.399399   0.102     29.232325]]
7 [[  0.102    -19.399399   0.102     29.232325]]
8 [[  0.102    -19.399399   0.102     29.232325]]
9 [[  0.102    -19.399399   0.102     29.232325]]


In [34]:
# Target Transition
for i in range(10) :
  #obs, rewards, dones, infos = self.env.step(clipped_actions)
  new_obs, rewards, dones, infos = target_env.step2(obs, clipped_actions)
  print (i, new_obs)

0 [[ 0.102      -0.09640235  0.102       0.42248276]]
1 [[ 0.102      -0.09640235  0.102       0.42248276]]
2 [[ 0.102      -0.09640235  0.102       0.42248276]]
3 [[ 0.102      -0.09640235  0.102       0.42248276]]
4 [[ 0.102      -0.09640235  0.102       0.42248276]]
5 [[ 0.102      -0.09640235  0.102       0.42248276]]
6 [[ 0.102      -0.09640235  0.102       0.42248276]]
7 [[ 0.102      -0.09640235  0.102       0.42248276]]
8 [[ 0.102      -0.09640235  0.102       0.42248276]]
9 [[ 0.102      -0.09640235  0.102       0.42248276]]
